In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import cv2
import torch
from PIL import Image
import os

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertForMultipleChoice, BertForMaskedLM, BertConfig, BertModel, AdamW
import random 

from sklearn.metrics import accuracy_score, f1_score
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

In [5]:
df_test = pd.read_csv('test_data.csv')
df_train = pd.read_csv('training_data.csv')

In [6]:
df_train.head()

,storyid,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [7]:
df_test.head()

,InputStoryid,InputSentence1,InputSentence2,InputSentence3,InputSentence4,RandomFifthSentenceQuiz1,RandomFifthSentenceQuiz2
0,5473848a-8394-4a9c-8e49-50e210524583,Rick grew up in a troubled household.,"He never found good support in family, and tur...",It wasn't long before Rick got shot in a robbery.,The incident caused him to turn a new leaf.,He joined a gang.,He is happy now.
1,dea6b57d-c8d3-4e15-8f0f-70c8e8d1fae0,Laverne needs to prepare something for her fri...,She decides to bake a batch of brownies.,She chooses a recipe and follows it closely.,Laverne tests one of the brownies to make sure...,Laverne doesn't go to her friend's party.,The brownies are so delicious Laverne eats two...
2,5137ead5-533d-4ab4-80e2-a06a0f3ae12c,Sarah had been dreaming of visiting Europe for...,She had finally saved enough for the trip.,She landed in Spain and traveled east across t...,She didn't like how different everything was.,Sarah decided that she preferred her home over...,Sarah then decided to move to Europe.
3,4b34b5c1-d3f7-4936-a02d-bbbc91583b09,Gina was worried the cookie dough in the tube ...,She was very happy to find she was wrong.,The cookies from the tube were as good as from...,Gina intended to only eat 2 cookies and save t...,Gina gave the cookies away at her church.,Gina liked the cookies so much she ate them al...
4,9d2206e5-131f-4c89-abdf-0cd8d33dd5d1,It was my final performance in marching band.,I was playing the snare drum in the band.,We played Thriller and Radar Love.,The performance was flawless.,I was very proud of my performance.,I was very ashamed of my performance.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")

The following code was used to finetune the BertForMaskedLM with pretrained bert-large-uncased. Data Collator was used for masking words from sentences with a probability of 15%. Do not run as it will take a long time to train and requires a good amount of recources. I am adding this just for demonstration. The fine-tuned model can be downloaded from here: https://drive.google.com/file/d/1WQOkb76dWoe9Tvaxb7V31UiKgLBswl--/view?usp=sharing

In [ ]:
# from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask

# class StoryDataset(Dataset):
#     def __init__(self, file_path, tokenizer, max_length):
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.data = self._load_data(file_path)

#     def _load_data(self, file_path):
#       df = pd.read_csv(file_path)
#       data = []
#       for index, row in df.iterrows():
#           if 'training' in file_path:
#               story = {
#                   'story_id': row['storyid'],
#                   'sentences': [row['sentence1'], row['sentence2'], row['sentence3'], row['sentence4'], row['sentence5']]
#               }
#           else:
#               story = {
#                   'story_id': row['InputStoryid'],
#                   'sentences': [row['InputSentence1'], row['InputSentence2'], row['InputSentence3'], row['InputSentence4']],
#                   'RandomFifthSentenceQuiz1': row['RandomFifthSentenceQuiz1'],
#                   'RandomFifthSentenceQuiz2': row['RandomFifthSentenceQuiz2']
#               }
#           data.append(story)
#       return data

#     def __len__(self):
#         return len(self.data)
 
#     def __getitem__(self, idx):
#       story = self.data[idx]
#       sentences = ' '.join(story['sentences'])
#       tokenized = self.tokenizer(sentences, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
      
#       return {
#           'input_ids': tokenized['input_ids'].squeeze(),
#           'token_type_ids': tokenized['token_type_ids'].squeeze(),
#           'attention_mask': tokenized['attention_mask'].squeeze()
#       }


# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# # tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# train_dataset = StoryDataset('training_data.csv', tokenizer, max_length=128)
# test_dataset = StoryDataset('test_data.csv', tokenizer, max_length=128)

# model_mlm = BertForMaskedLM.from_pretrained('bert-large-uncased')
# model_mlm.to(device)
# model_mlm.train()


# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
# # data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm_probability=0.15)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=data_collator)

# optimizer = AdamW(model_mlm.parameters(), lr=5e-5)


# # Training loop. Remember to change the model accordingly

# num_epochs = 10
# total_steps = len(train_loader) * num_epochs
# warmup_steps = int(total_steps * 0.1)

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# def masked_metrics(preds, labels, ignore_index):
#     batch_size, seq_len, _ = preds.size()
#     preds_flat = preds.argmax(dim=-1).view(batch_size, seq_len)
#     labels_flat = labels.view(batch_size, seq_len)
    
#     mask = (labels_flat != ignore_index)

#     # print("preds_flat shape:", preds_flat.shape)
#     # print("labels_flat shape:", labels_flat.shape)
#     # print("mask shape:", mask.shape) 

#     mask = mask.bool()  # Convert the mask to a boolean tensor
#     preds_flat = preds_flat[mask]
#     labels_flat = labels_flat[mask]

#     return accuracy_score(labels_flat.cpu().numpy(), preds_flat.cpu().numpy())

# def masked_metrics(preds, labels, ignore_index):
#     batch_size, seq_len, _ = preds.size()
#     preds_flat = preds.argmax(dim=-1).view(batch_size, seq_len)
#     labels_flat = labels.view(batch_size, seq_len)
    
#     mask = (labels_flat != ignore_index)
#     mask = mask.bool()  # Convert the mask to a boolean tensor
#     preds_flat = preds_flat[mask]
#     labels_flat = labels_flat[mask]

#     acc = accuracy_score(labels_flat.cpu().numpy(), preds_flat.cpu().numpy())
#     f1 = f1_score(labels_flat.cpu().numpy(), preds_flat.cpu().numpy(), average='macro')

#     return acc, f1



# for epoch in tqdm(range(num_epochs), desc='Epochs'):
#     epoch_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
#     running_accuracy = 0
#     running_f1 = 0
#     running_loss = 0
#     num_batches = 0

#     for batch in epoch_iterator:
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)

#         optimizer.zero_grad()
    
#         outputs = model_mlm(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         scheduler.step()

        
#         accuracy, f1 = masked_metrics(outputs.logits, labels, tokenizer.pad_token_id)
#         running_accuracy += accuracy
#         running_f1 += f1
#         running_loss += loss.item()
#         num_batches += 1

       
#         epoch_iterator.set_postfix({'Loss': loss.item(), 'Accuracy': accuracy})
    
#     epoch_loss = running_loss / num_batches
#     epoch_accuracy = running_accuracy / num_batches
#     epoch_f1 = running_f1 / num_batches

#     print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.4f} | F1 Score: {epoch_f1:.4f}")

# The fine-tuned model can be downloaded from the link bellow 
# https://drive.google.com/file/d/1WQOkb76dWoe9Tvaxb7V31UiKgLBswl--/view?usp=sharing

In [9]:
# copy finetuned BertForMaskedLM from drive.
!cp -r /content/drive/MyDrive/Deep_Learning_COM3025/bert_large_mlm2.pt /content/

In [ ]:
from transformers import BertForMaskedLM
model_mlm = BertForMaskedLM.from_pretrained('bert-large-uncased')

model_mlm.load_state_dict(torch.load('bert_large_mlm2.pt'))

model_mlm.to(device)

### Bellow, the weights of previously fine-tuned BertForMaskedLM are transfered to BertForMultipleChoice and fine-tuned further for one epoch. To train this model I am using the last sentence from the training data as a correct ending and then choose a random sentence from another story as an incorect ending. The model is then trained for one epoch.

In [11]:
from transformers import BertTokenizer, BertForMultipleChoice, BertConfig, BertModel
import random 

class BertForMultipleChoiceCustom(nn.Module):
    def __init__(self, bert_model):
        super(BertForMultipleChoiceCustom, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1024, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
      num_choices = input_ids.size(1)
      input_ids = input_ids.view(-1, input_ids.size(-1))
      attention_mask = attention_mask.view(-1, attention_mask.size(-1))
      token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1))

      outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
      last_hidden_state = outputs.last_hidden_state
      pooled_output = torch.mean(last_hidden_state, dim=1)
      pooled_output = self.dropout(pooled_output)
      logits = self.classifier(pooled_output)
      reshaped_logits = logits.view(-1, num_choices)

      return reshaped_logits


class StoryDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length):
      self.tokenizer = tokenizer
      self.max_length = max_length
      self.data = self._load_data(file_path)

    def _load_data(self, file_path):
      df = pd.read_csv(file_path)
      data = []
      for index, row in df.iterrows():
          if 'training' in file_path:
              story = {
                  'story_id': row['storyid'],
                  'sentences': [row['sentence1'], row['sentence2'], row['sentence3'], row['sentence4']],
                  'ending': row['sentence5']
              }
          data.append(story)
      return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      story = self.data[idx]
      context = ' '.join(story['sentences'])

      # Correct ending
      correct_ending = story['ending']

      # Randomly choose an incorrect ending from another story
      wrong_story = random.choice(self.data)
      while wrong_story['story_id'] == story['story_id']:
          wrong_story = random.choice(self.data)
      wrong_ending = wrong_story['sentences'][3]

      # Randomize the order of the endings
      endings = [(correct_ending, 0), (wrong_ending, 1)]
      random.shuffle(endings)
      endings, labels = zip(*endings)

      # Tokenize both endings
      tokenized_endings = [
          self.tokenizer(
              context,
              ending,
              padding='max_length',
              truncation=True,
              max_length=self.max_length,
              return_tensors='pt',
          )
          for ending in endings
      ]

      
      input_ids = torch.stack([t['input_ids'] for t in tokenized_endings], dim=0)
      token_type_ids = torch.stack([t['token_type_ids'] for t in tokenized_endings], dim=0)
      attention_mask = torch.stack([t['attention_mask'] for t in tokenized_endings], dim=0)


      return {
          'input_ids': input_ids,
          'token_type_ids': token_type_ids,
          'attention_mask': attention_mask,
          'labels': torch.tensor(labels[0], dtype=torch.long)  # The correct answer's index
      }


max_length = 128
batch_size = 32
epochs = 1
lr = 3e-5

tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

# Transfer the weights from the MLM model to a MultipleChoiceCustom model

model_mc = BertForMultipleChoiceCustom(model_mlm.bert)

model_mc.to(device)






# Load the data
file_path = "training_data.csv"

# Create the custom dataset and dataloader
dataset = StoryDataset(file_path, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=batch_size)



# Optimizer
optimizer = AdamW(model_mc.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()


# Fine-tuning loop
for epoch in range(epochs):
    model_mc.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # outputs = model_mc(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
        logits = model_mc(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

        # loss = outputs.loss
        loss = criterion(logits, labels)


        total_loss += loss.item()

        # Calculate F1 score
        preds = torch.argmax(logits, dim=1).cpu().detach().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().detach().numpy())

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(dataloader)
    f1 = f1_score(all_labels, all_preds, average='macro')

    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_train_loss}, F1 Score: {f1}")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/1:   0%|          | 0/1646 [00:00<?, ?it/s]

Epoch 1/1 - Loss: 0.02753563823366281, F1 Score: 0.9908285059222647


In [12]:
torch.save(model_mc.state_dict(), 'bert_large_mlm3.pt')

In [ ]:
# copy to drive
!cp -r /content/bert_large_mlm3.pt /content/drive/MyDrive/Deep_Learning_COM3025/

In [13]:
#3 make predictions using Bertformultiplechoice
import csv

class BertForMultipleChoiceCustom(nn.Module):
    def __init__(self, bert_model):
        super(BertForMultipleChoiceCustom, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(1024, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        num_choices = input_ids.size(1)
        input_ids = input_ids.view(-1, input_ids.size(-1))
        attention_mask = attention_mask.view(-1, attention_mask.size(-1))
        token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1))

        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        reshaped_logits = logits.view(-1, num_choices)

        return reshaped_logits




def generate_prediction(input_story, option1, option2, model, tokenizer):
    max_length = 128
    prompt = input_story
    choice0 = option1
    choice1 = option2

    encoding = tokenizer([prompt, prompt], [choice0, choice1], return_tensors="pt", padding=True, max_length=max_length, truncation=True)
    encoding = {k: v.unsqueeze(0).to(device) for k, v in encoding.items()}

    with torch.no_grad():
        logits = model(**encoding)
        logits = logits.squeeze()

    answer = 0 if logits[0] > logits[1] else 1
    return answer






# Transfer the weights from the MLM model to a MultipleChoice model. 
# If using just the pretrained MC model without fine tuning, uncoment the next 4 lines

# bert_model = BertModel.from_pretrained('bert-large-cased')
# bert_model = model_mc.bert

# model_mc = BertForMultipleChoiceCustom(bert_model)
# model_mc.to(device)
model_mc.eval()



with open('test_data.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    results = []

    for row in reader:
        story_id = row['InputStoryid']
        context = ' '.join([row['InputSentence1'], row['InputSentence2'], row['InputSentence3'], row['InputSentence4']])
        option1 = row['RandomFifthSentenceQuiz1']
        option2 = row['RandomFifthSentenceQuiz2']

        answer = generate_prediction(context, option1, option2, model_mc, tokenizer)
        results.append({'InputStoryid': story_id, 'AnswerRightEnding': answer})

# Write the results
with open('predictions.csv', 'w', newline='') as csvfile:
    fieldnames = ['InputStoryid', 'AnswerRightEnding']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)

In [14]:
# best prediction 64% : fine tuned Bert large for masked Lm, 10 epochs with 5e-5 lr rate then transfer the weights to bert for multiple choice and fine tune for one epoch.
!python3 /usr/local/bin/kaggle competitions submit -m $URN -c com3025-2023-challenge2 -f predictions.csv

100% 61.4k/61.4k [00:00<00:00, 97.3kB/s]
Successfully submitted to COM3025-2023-Challenge2